# BOLETIM DO COMÉRCIO EXTERIOR


## Variáveis Globais

In [1]:
nome_planilha = 'BCE 2024.03 - São João del-Rei'
nome_guia     = 'IMPORTAÇÃO - Países'

## Importando bibliotecas

In [2]:
# General
import pandas as pd

# Graphs
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Google
import gspread
from google.colab import auth
from google.auth import default

## Autenticando no Google Planilhas

In [3]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Importando dados das planilhas

In [4]:
planilha = gc.open(nome_planilha)

guia = planilha.worksheet(nome_guia)

dados = guia.get_all_values()

# Converter 'dados' para um DataFrame
sjdr_imports = pd.DataFrame(dados[1:], columns=dados[0])

In [5]:
sjdr_imports.head()

,,,,,,
0,,País,Valor (US$),,País,Valor (US$)
1,,China,"$ 2.416.549,00",,China,"$ 2.416.549,00"
2,,Singapura,"$ 1.383.182,00",,Singapura,"$ 1.383.182,00"
3,,Espanha,"$ 583.589,00",,Espanha,"$ 583.589,00"
4,,França,"$ 309.948,00",,França,"$ 309.948,00"


### Formatando os dados

In [6]:
# Definir a primeira linha como cabeçalho
sjdr_imports.columns = sjdr_imports.iloc[0]
sjdr_imports = sjdr_imports.iloc[1:].reset_index(drop=True)

# Deletar colunas que não vou utilizar
sjdr_imports = sjdr_imports.iloc[0:, :3]

# Remover a primeira coluna
sjdr_imports = sjdr_imports.drop(sjdr_imports.columns[0], axis=1)

# Renomear colunas
sjdr_imports = sjdr_imports.rename(columns={
    sjdr_imports.columns[0]: 'country',
    sjdr_imports.columns[1]: 'value',
})

# Substituir valores por vazio
sjdr_imports['value'] = sjdr_imports['value'].str.replace('\$', '', regex=True)
sjdr_imports['value'] = sjdr_imports['value'].str.replace(',00', '', regex=True)
sjdr_imports['value'] = sjdr_imports['value'].str.replace('.', '', regex=False)

# Remover espaços em branco e caracteres não numéricos
sjdr_imports['value'] = sjdr_imports['value'].str.replace('[^\d.]', '', regex=True).str.strip()

# Converter os dados para números (float)
sjdr_imports['value'] = pd.to_numeric(sjdr_imports['value'], errors='coerce')

In [7]:
# Vizualizar o dataframe
sjdr_imports.head()

,country,value
0,China,2416549
1,Singapura,1383182
2,Espanha,583589
3,França,309948
4,Turquia,126910


In [8]:
# Vizualizar informações do dataframe
sjdr_imports['value'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 11 entries, 0 to 10
Series name: value
Non-Null Count  Dtype
--------------  -----
11 non-null     int64
dtypes: int64(1)
memory usage: 216.0 bytes


### Tabela com código dos países

In [9]:
planilha = gc.open('Lista com códigos dos países')

guia = planilha.worksheet('Lista')

dados = guia.get_all_values()

# Converter 'dados' para um DataFrame
countries = pd.DataFrame(dados[1:], columns=dados[0])

# Deletar colunas
countries.drop(['country', 'alpha-2-code', 'numeric'], axis=1, inplace=True)

# Renomear colunas
countries = countries.rename(columns={
    countries.columns[0]: 'country',
    countries.columns[1]: 'code',
})

### Mesclando os dados para plotar

In [10]:
# Mesclando os dados de exportação com códigos dos países
imports = pd.merge(countries, sjdr_imports, how='inner', on='country', indicator=False)

In [11]:
# Vizualizar o dataframe
imports.head()

,country,code,value
0,Alemanha,DEU,45376
1,China,CHN,2416549
2,Espanha,ESP,583589
3,Estados Unidos,USA,5232
4,França,FRA,309948


In [12]:
# Vizualizar informações do dataframe
imports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  11 non-null     object
 1   code     11 non-null     object
 2   value    11 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 392.0+ bytes


## Plotando o gráfico

In [13]:
data = go.Choropleth(
    locations = imports['code'],
    z = imports['value'],
    text = imports['country'],
    colorscale = 'redor',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = 'U$',
    colorbar_title = 'Valores<br> em Milhões (U$)',
    showlegend=True,
    showscale=True,
)

layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig = go.Figure(data=data, layout=layout)

fig = fig.update_layout(
    font_family="Arial",
    font_color="#FFFFFF", # 004651
    title_font_family="Arial",
    title_font_color="#FFFFFF",
    legend_title_font_color="#FFFFFF",
    width=900,
    height=500,
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular',
        bgcolor= '#F4F4F0',
        landcolor='rgba(206,206,206,1)',
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)

fig.update_geos(fitbounds="locations", visible=True)


fig.show()